In [1]:
from ipython_demo_support.utils import PointCloudData, SegmentationModel

/home/thor/anaconda3/envs/Lidar_Seg/lib/python3.10/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/home/thor/anaconda3/envs/Lidar_Seg/lib/python3.10/site-packages/spconv/pytorch/functional.py:97: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/thor/anaconda3/envs/Lidar_Seg/lib/python3.10/site-packages/spconv/pytorch/functional.py:163: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/thor/anaconda3/envs/Lidar_Seg/lib/python3.10/site-packages/spconv/pytorch/functional.py:243: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is de

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Load the point cloud data
point_cloud = PointCloudData()

# KITTI example
point_cloud_data = point_cloud.load_point_cloud("data_examples/KITTI/000000.bin")

# DALES example
# point_cloud_data = point_cloud.load_point_cloud("data_examples/DALES/5080_54400_42.npy")

# ScanNet example
# point_cloud_data = point_cloud.load_point_cloud("data_examples/ScanNet/scene0046_02")


# Load the model
model = SegmentationModel(checkpoint_path="checkpoints/SNAP_C.pth", domain="Outdoor")

# Intialize the model and run the model backbone to extract point features
point_cloud_data = model.intialize_pointcloud(point_cloud_data)
model.extract_backbone_features(point_cloud_data)

# Specify a prompt point
# prompt_points = [[[x1, y1, z1], [x2, y2, z2], ...]]  # List of prompt pointe, Shape -> M, P, 3 (M = number of objects, P = number of clicks on each object)
# The prompt points should be in the format [x, y, z] and should be in the same coordinate system as the point cloud data
prompt_points = [[[-15.1562, -17.0000,  12.7000]]] # Prompt for aerial 

# Run segmentation
masks, text_labels, iou_scores, text_tokens = model.segment(point_cloud_data, prompt_points)

# Visualize the results
model.visualize_results(point_cloud_data, masks, text_labels, iou_scores)

Intensity data found
Min and max are:  0.0 0.99
Number of trainable params: 49397965
Loading checkpoint from checkpoints/SNAP_C.pth


/home/thor/Projects/SNAP/ipython_demo_support/utils.py:503: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789122112/work/torch/csrc/utils/tensor_new.cpp:278.)
  prompt_points_tensor = torch.tensor(processed_points, dtype=torch.float32, device=self.device).unsqueeze(1)


Widget(value='<iframe src="http://localhost:35133/index.html?ui=P_0x7162c1a8ff40_0&reconnect=auto" class="pyvi…

In [5]:
# Segmentation Everything
masks, text_labels, iou_scores, generated_prompts = model.segment_everything(point_cloud_data, max_iters=3, initial_voxel_size=10) # Increase max_iters for better coverage at the cost of slightly slower speed. 

# Visualize the results
model.visualize_results(point_cloud_data, masks, text_labels, iou_scores)


--- Iteration 1 ---
Points remaining to be segmented: 88763
Using Voxel Size: 10.00
Generated 134 new prompt points.

--- Iteration 2 ---
Points remaining to be segmented: 13155
Using Voxel Size: 5.00
Generated 170 new prompt points.

--- Iteration 3 ---
Points remaining to be segmented: 3091
Using Voxel Size: 2.50
Generated 238 new prompt points.

--- Post-Processing ---
Running NMS on 142 masks...
Kept 66 final masks after NMS.


Widget(value='<iframe src="http://localhost:35133/index.html?ui=P_0x7162a94ec370_2&reconnect=auto" class="pyvi…

In [7]:
# Visualize the segment everything masks based on classes from class vocabulary. (Specified class must be from class vocab)
indices = model.find_masks_by_label(text_labels, "car")
selected_masks = [masks[i] for i in indices]
selected_text_labels = [text_labels[i] for i in indices]
selected_iou_scores = [iou_scores[i] for i in indices]

# Visualize the results
model.visualize_results(point_cloud_data, selected_masks, selected_text_labels, selected_iou_scores)

Found 30 mask(s) with the label 'car' at indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 22, 23, 25, 27, 29, 35, 37, 39, 51, 54]


Widget(value='<iframe src="http://localhost:35133/index.html?ui=P_0x7162a090c5e0_4&reconnect=auto" class="pyvi…

In [8]:
# Segment based on text (Text string need not be from the vocabulary - Go crazy!)
indices = model.find_mask_from_text("vehicle", 0.4)
selected_masks = [masks[i] for i in indices]
selected_text_labels = [text_labels[i] for i in indices]
selected_iou_scores = [iou_scores[i] for i in indices]

# Visualize the results
model.visualize_results(point_cloud_data, selected_masks, selected_text_labels, selected_iou_scores)

Found 24 significant matches for 'vehicle'.


Widget(value='<iframe src="http://localhost:35133/index.html?ui=P_0x7162a090ccd0_5&reconnect=auto" class="pyvi…